In [1]:
# You should change your bucket name below. See the book for more details
(https://www.amazon.com/Data-Analytics-Google-Cloud-Hands-ebook/dp/B087XZZ2C6/)
data = spark.read.csv("gs://rs-movielens-2/ratings.csv", header=True, inferSchema=True)

In [2]:
data.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [3]:
print(data.rdd.take(2)) # viewing 2 rows of the raw rdd

# Convert your dataframe into an RDD, and then into a list
ratings = data.rdd.map(list)
ratings.take(2)

[Row(userId=1, movieId=1, rating=4.0, timestamp=964982703), Row(userId=1, movieId=3, rating=4.0, timestamp=964981247)]


[[1, 1, 4.0, 964982703], [1, 3, 4.0, 964981247]]

In [5]:
# Import the Rating object
from pyspark.mllib.recommendation import Rating
# Convert the data into Rating objects
ratings_data = ratings.map(lambda line: Rating(int(line[0]), int(line[1]), float(line[2])));
# This is what a Rating object looks like
ratings_data.take(2)

[Rating(user=1, product=1, rating=4.0), Rating(user=1, product=3, rating=4.0)]

In [6]:
# Split the data into training and test, in 80-20% ratio
training_data, test_data = ratings_data.randomSplit([0.8,0.2]);

In [7]:
# Import the ALS method
from pyspark.mllib.recommendation import ALS
# Build the model based on the training data, with tank = 10 and iterations = 10
model = ALS.train(training_data, rank=10, iterations=10)

In [8]:
# Drop the ratings column
testdata_nr = test_data.map(lambda p: (p[0],p[1]))
testdata_nr.take(2)

[(1, 3), (1, 151)]

In [9]:
# Predict the model  
predictions = model.predictAll(testdata_nr)

In [10]:
# Print the first rows of the RDD
predictions.take(2)

[Rating(user=414, product=1084, rating=4.2084196107141425),
 Rating(user=294, product=1084, rating=2.79318157043616)]

In [11]:
# Prepare ratings data
ratings_kv = ratings_data.map(lambda r: ((r[0],r[1]),r[2]));
ratings_kv.take(2)

[((1, 1), 4.0), ((1, 3), 4.0)]

In [12]:
# Prepare predictions data
predictions_kv = predictions.map(lambda r: ((r[0],r[1]),r[2]))
predictions_kv.take(2)

[((414, 1084), 4.2084196107141425), ((294, 1084), 2.79318157043616)]

In [21]:
# Join the ratings data with predictions data
ratings_predictions = ratings_kv.join(predictions_kv)
ratings_predictions.take(5)

[((1, 3), (4.0, 3.211911768374442)),
 ((1, 151), (5.0, 3.2276032841269178)),
 ((1, 457), (5.0, 4.7321496727891255)),
 ((1, 919), (5.0, 3.9240397529605335)),
 ((1, 1089), (5.0, 4.7363555272243305))]

In [25]:
# Convert RDD to Dataframe
# First, create a clean version of the ratings_predictions RDD
ratings_predictions_clean = ratings_predictions.map(lambda r: (r[0][0], r[0][1], r[1][0], r[1][1]))
# Next, convert this clean version to a PySpark dataframe
df_ratings_predictions = ratings_predictions_clean.toDF()
# Print the dataframe schema - just for your information
df_ratings_predictions.printSchema()
# Print some records in the dataframe - just for your information
df_ratings_predictions.show(5)
# Save the dataframe df_ratings_predictions into a single csv file using "coalesce" command
df_ratings_predictions.coalesce(1).write.format('com.databricks.spark.csv').save('gs://rs-movielens-2/ratings_predictions1.csv')

root
 |-- _1: long (nullable = true)
 |-- _2: long (nullable = true)
 |-- _3: double (nullable = true)
 |-- _4: double (nullable = true)

+---+----+---+------------------+
| _1|  _2| _3|                _4|
+---+----+---+------------------+
|  1|   3|4.0| 3.211911768374442|
|  1| 151|5.0|3.2276032841269178|
|  1| 457|5.0|4.7321496727891255|
|  1| 919|5.0|3.9240397529605335|
|  1|1089|5.0|4.7363555272243305|
+---+----+---+------------------+
only showing top 5 rows



In [34]:
# Calculate and print MAE
MAE = ratings_predictions.map(lambda r: abs(r[1][0] - r[1][1])).mean()
print("Mean Absolute Error of the model for the test data = {:.4f}".format(MAE))

Mean Absolute Error of the model for the test data = 0.8411


In [33]:
# Calculate and print MSE
MSE = ratings_predictions.map(lambda r: (r[1][0] - r[1][1])**2).mean()
print("Mean Squared Error of the model for the test data = {:.4f}".format(MSE))

Mean Squared Error of the model for the test data = 1.2746


In [32]:
# Calculate and print RMSE
#import org.apache.spark.sql.functions.sqrt
import math
RMSE = MSE**(1/2)
print("Root Mean Squared Error of the model for the test data = {:.4f}".format(RMSE))

Root Mean Squared Error of the model for the test data = 1.1290
